https://huggingface.co/AnyaSchen/vit-rugpt3-large-poetry-ft/blob/main/README.md?code=true

In [1]:
from PIL import Image
import requests, torch
from transformers import AutoTokenizer, VisionEncoderDecoderModel, ViTImageProcessor

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU device: {device}')
else:
    device = torch.device('cpu')
    print(f'GPU is not available, using CPU device {device}')

GPU is not available, using CPU device cpu


In [3]:
def generate_poetry(fine_tuned_model, image, tokenizer, author):
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    # Encode author's name and prepare as input to the decoder
    author_input = f"<bos> {author} <sep>"
    decoder_input_ids = tokenizer.encode(author_input, return_tensors="pt").to(device)
    # Generate the poetry with the fine-tuned VisionEncoderDecoder model
    generated_tokens = fine_tuned_model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=300,
        num_beams=3,
        top_p=0.8,
        temperature=2.0,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    # Decode the generated tokens
    generated_poetry = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    generated_poetry = generated_poetry.split(f'{author}')[-1]
    return generated_poetry

In [4]:
path = 'AnyaSchen/vit-rugpt3-large-poetry-ft'
fine_tuned_model = VisionEncoderDecoderModel.from_pretrained(path).to(device)
feature_extractor = ViTImageProcessor.from_pretrained(path)
tokenizer = AutoTokenizer.from_pretrained(path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.99k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

In [6]:
pictures = ['https://anandaindia.org/wp-content/uploads/2018/12/happy-man.jpg',
            'https://www.mentoday.ru/upload/img_cache/a83/a83e6e1aadcf2c5ee270132f7553cf28_ce_1500x1000x0x0_cropped_666x444.jpg',
            'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDiVYBOgBZcfd5c8eyar-fZAXnUuL6RVbTimPzB_9Few&s']

In [10]:
poems = []

In [11]:
for pic in pictures:
    pic = Image.open(requests.get(pic, stream=True).raw)
    poems.append(generate_poetry(fine_tuned_model, pic, tokenizer, 'Пушкин'))
    poems.append(generate_poetry(fine_tuned_model, pic, tokenizer, 'Есенин'))

In [13]:
for i in poems:
    print(i)
    print('-----------------')

  Спеши, моя мечта!
Я жду тебя в тени густой
Места спокойного, священного,
Где наслажденье в тишине
Впервые встретилося мне.
Где ж оно?О, скажи мне,
Где наслажденье встречалося мне?
Мне сладко вспомнить его следы
На сердце тихом, как ныне
Вечерние лучи заката.
 
-----------------
  Говорю тебе — не пой молебствия
Тому, кто любовью и силой
Зарешал все свои тревоги
И поверил всем тайнам мирозданья.
 
-----------------
  Желудок мой справедлив:
У меня весной
Желудок так же верен,
Как в зимнюю стужу.
 
-----------------
  И потому в желтом свитере
И с лысой головой,
Что хочет всем нравиться,
Всем нравиться, собою.
 
-----------------
  Цветы красноречивы говорят,
Я в них сердца нахожу,
В них мысли глубокие горят,
И сладостно читать их страницы,
Когда за чашей круговой
Во мраке ночи долу склонясь,
Твой взор в забвенье увлеку
За пиршеством младости златой,
Где всё сражалось, волновалось,
Шумело, пело и кричало,
И ты, судьбой хранимый,
Один в кругу друзей прекрасных
Сверкал звездою златой
 
-